Import Genre data from CSV

In [87]:
import pandas as pd
movie_genre = pd.read_csv("./data/MovieGenre.csv",
                          sep=",", encoding='unicode_escape', index_col=None)
movie_genre.dropna(subset=['Genre'], inplace=True)
display(movie_genre.head())
display(movie_genre.dtypes)
poster_urls = movie_genre['Poster']



,imdbId,Imdb Link,Title,IMDB Score,Genre,Poster
0,114709,http://www.imdb.com/title/tt114709,Toy Story (1995),8.3,Animation|Adventure|Comedy,https://images-na.ssl-images-amazon.com/images...
1,113497,http://www.imdb.com/title/tt113497,Jumanji (1995),6.9,Action|Adventure|Family,https://images-na.ssl-images-amazon.com/images...
2,113228,http://www.imdb.com/title/tt113228,Grumpier Old Men (1995),6.6,Comedy|Romance,https://images-na.ssl-images-amazon.com/images...
3,114885,http://www.imdb.com/title/tt114885,Waiting to Exhale (1995),5.7,Comedy|Drama|Romance,https://images-na.ssl-images-amazon.com/images...
4,113041,http://www.imdb.com/title/tt113041,Father of the Bride Part II (1995),5.9,Comedy|Family|Romance,https://images-na.ssl-images-amazon.com/images...


imdbId          int64
Imdb Link      object
Title          object
IMDB Score    float64
Genre          object
Poster         object
dtype: object

Get all existing genres as list

In [86]:
def flatten(listToFlatten):
    return [item for sublist in listToFlatten for item in sublist]

genres = movie_genre["Genre"].str.split('|').array.tolist()
distinct_genres = list(set(flatten(genres)))
distinct_genres


['Sci-Fi',
 'Comedy',
 'Mystery',
 'Game-Show',
 'Animation',
 'History',
 'Documentary',
 'Horror',
 'Sport',
 'News',
 'Fantasy',
 'Family',
 'Musical',
 'Talk-Show',
 'Crime',
 'Reality-TV',
 'Film-Noir',
 'Thriller',
 'Adventure',
 'Biography',
 'Action',
 'Western',
 'War',
 'Adult',
 'Music',
 'Drama',
 'Short',
 'Romance']

Create Array with all ids with all genres and place 0 and 1 (1 when movie has Genre)
e.q
ID  Action Comedy Romance   ....
123 1       1      0        ....

In [ ]:
ids_with_genre = []


Load Sample Images

In [21]:
import os
import cv2
samples_folder = "./data/SampleMoviePosters"
files = os.listdir(samples_folder)
sample_images = pd.DataFrame(columns=["imdbId", "img"])
sample_images.set_index("imdbId", inplace=True)
for file in files:
    id = int(file.replace(".jpg", ""))
    img = cv2.imread(os.path.join(samples_folder, file))
    sample_images.loc[id, "img"] = img
sample_images = pd.merge(sample_images, movie_genre,
                  left_index=True, right_on="imdbId")
display(sample_images.head())

,img,Imdb Link,Title,IMDB Score,Genre,Poster
imdbId,,,,,,
2461,"[[[149, 122, 138], [157, 130, 146], [159, 133,...",http://www.imdb.com/title/tt2461,Richard III (1912),5.7,Drama,https://images-na.ssl-images-amazon.com/images...
24886,"[[[51, 63, 35], [72, 81, 55], [60, 58, 40], [5...",http://www.imdb.com/title/tt24886,The Big Bad Wolf (1934),7.2,Animation|Comedy|Family,https://images-na.ssl-images-amazon.com/images...
17162,"[[[63, 88, 228], [64, 88, 230], [62, 82, 229],...",http://www.imdb.com/title/tt17162,Moana (1926),6.9,Documentary,https://images-na.ssl-images-amazon.com/images...
25239,"[[[151, 215, 246], [150, 214, 245], [150, 214,...",http://www.imdb.com/title/tt25239,Here Is My Heart (1934),7.1,Musical|Comedy,https://images-na.ssl-images-amazon.com/images...
21345,"[[[30, 31, 27], [30, 31, 27], [30, 31, 27], [3...",http://www.imdb.com/title/tt21345,The Sea Bat (1930),5.6,Action|Romance|Thriller,https://images-na.ssl-images-amazon.com/images...


Download posters

In [2]:
import os
import urllib
images_path = "./data/posters/"
mkdir_cmd = "mkdir -p " + images_path
os.system(mkdir_cmd)
failed_downloads = []
for poster in poster_urls.itertuples():
    file_name = str(poster.imdbId) + ".jpg"
    if os.path.isfile(images_path + file_name):
        continue
    try:
        resource = urllib.request.urlretrieve(
            poster.Poster, images_path + file_name)
    except:
        failed_downloads.append(str(poster.Poster))
        continue
print("Failed downloads: " + str(failed_downloads))
if len(failed_downloads) == 0:
    print("All images downloaded successfully")


In [23]:
import tensorflow as tf
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)
model.evaluate(x_test, y_test)


2022-05-18 21:12:15.537650: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:961] could not open file to read NUMA node: /sys/bus/pci/devices/0000:26:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-05-18 21:12:15.609348: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:961] could not open file to read NUMA node: /sys/bus/pci/devices/0000:26:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-05-18 21:12:15.609729: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:961] could not open file to read NUMA node: /sys/bus/pci/devices/0000:26:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-05-18 21:12:15.611335: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate

Epoch 1/5
1875/1875 [==============================] - 10s 5ms/step - loss: 0.2913 - accuracy: 0.9145
Epoch 2/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.1419 - accuracy: 0.9579
Epoch 3/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.1076 - accuracy: 0.9675
Epoch 4/5
1875/1875 [==============================] - 10s 5ms/step - loss: 0.0881 - accuracy: 0.9725
Epoch 5/5
313/313 [==============================] - 1s 4ms/step - loss: 0.0743 - accuracy: 0.9791


[0.07427120953798294, 0.9790999889373779]

Persist Model

In [25]:
model.save("./data/model")

INFO:tensorflow:Assets written to: ./data/model/assets


Load Model

In [27]:
model = tf.keras.models.load_model('./data/model')

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 128)               100480    
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 10)                1290      
                                                                 
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________
